<a href="https://www.kaggle.com/code/ibkya12/map-0-98-leashmp-competition?scriptVersionId=180624335" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 40.0 MB/s eta 0:00:00


In [2]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 60.0 MB/s eta 0:00:00


## Data Preparation

The training and testing data paths are defined for the .parquet files. We use duckdb to scan search through the large training sets. Just to get started lets sample out an equal number of positive and negatives. 

This query selects an equal number of samples where binds equals 0 (non-binding) and 1 (binding), limited to 30,000 each, to avoid model bias towards a particular class.

In [3]:
import duckdb
import pandas as pd

train_path = '/kaggle/input/leash-predict-chemical-bindings/train.parquet'
test_path = '/kaggle/input/leash-predict-chemical-bindings/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 200000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 200000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,158060925,O=C(Nc1cc(Br)c(F)cc1C(=O)O)OCC1c2ccccc2-c2ccccc21,CCC#CCN,Nc1nc2cc([N+](=O)[O-])ccc2[nH]1,CCC#CCNc1nc(Nc2nc3cc([N+](=O)[O-])ccc3[nH]2)nc...,BRD4,0
1,264561853,O=C(O)C[C@H](Cc1c(F)c(F)c(F)c(F)c1F)NC(=O)OCC1...,Nc1ccc(F)cc1F,CC(C)NC(=O)NCCN.Cl.Cl,CC(C)NC(=O)NCCNc1nc(Nc2ccc(F)cc2F)nc(N[C@H](CC...,HSA,0
2,156279215,O=C(Nc1cc(-n2cccn2)ccc1C(=O)O)OCC1c2ccccc2-c2c...,Cl.Cl.NC[C@@H]1CCO[C@H]1c1cn[nH]c1,COCc1ccccc1CN,COCc1ccccc1CNc1nc(NC[C@@H]2CCO[C@H]2c2cn[nH]c2...,sEH,0
3,7920493,C=CCC[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,COc1cncc(N)c1,Cc1cnc(O)c(N)c1,C=CCC[C@@H](Nc1nc(Nc2cncc(OC)c2)nc(Nc2cc(C)cnc...,HSA,0
4,54720488,Cc1ccc(C(=O)O)cc1NC(=O)OCC1c2ccccc2-c2ccccc21,COc1cc(F)c(Cl)cc1N,NCCC(=O)N1CCN(c2ccccn2)CC1,COc1cc(F)c(Cl)cc1Nc1nc(NCCC(=O)N2CCN(c3ccccn3)...,sEH,0


## Feature Preprocessing

Lets grab the smiles for the fully assembled molecule `molecule_smiles` and generate ecfps for it. We could choose different radiuses or bits, but 2 and 1024 is pretty standard.

In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

## Train Model

In [6]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,molecule,ecfp
0,158060925,O=C(Nc1cc(Br)c(F)cc1C(=O)O)OCC1c2ccccc2-c2ccccc21,CCC#CCN,Nc1nc2cc([N+](=O)[O-])ccc2[nH]1,CCC#CCNc1nc(Nc2nc3cc([N+](=O)[O-])ccc3[nH]2)nc...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x789a2d245b60>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,264561853,O=C(O)C[C@H](Cc1c(F)c(F)c(F)c(F)c1F)NC(=O)OCC1...,Nc1ccc(F)cc1F,CC(C)NC(=O)NCCN.Cl.Cl,CC(C)NC(=O)NCCNc1nc(Nc2ccc(F)cc2F)nc(N[C@H](CC...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x789a5d78e5e0>,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,156279215,O=C(Nc1cc(-n2cccn2)ccc1C(=O)O)OCC1c2ccccc2-c2c...,Cl.Cl.NC[C@@H]1CCO[C@H]1c1cn[nH]c1,COCc1ccccc1CN,COCc1ccccc1CNc1nc(NC[C@@H]2CCO[C@H]2c2cn[nH]c2...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x789a5d78e650>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,7920493,C=CCC[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,COc1cncc(N)c1,Cc1cnc(O)c(N)c1,C=CCC[C@@H](Nc1nc(Nc2cncc(OC)c2)nc(Nc2cc(C)cnc...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x789a5d78e6c0>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,54720488,Cc1ccc(C(=O)O)cc1NC(=O)OCC1c2ccccc2-c2ccccc21,COc1cc(F)c(Cl)cc1N,NCCC(=O)N1CCN(c2ccccn2)CC1,COc1cc(F)c(Cl)cc1Nc1nc(NCCC(=O)N2CCN(c3ccccn3)...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x789a5d78e730>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
from xgboost import XGBClassifier

In [9]:
xgb = XGBClassifier()

xgb_model = xgb.fit(X_train , y_train)

y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.97


# Best Score received with CatBoostClassifier

In [10]:
from catboost import CatBoostClassifier
catb_model = CatBoostClassifier().fit(X_train, y_train)
y_pred_proba = catb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Learning rate set to 0.120958
0:	learn: 0.6509518	total: 219ms	remaining: 3m 38s
1:	learn: 0.6185208	total: 386ms	remaining: 3m 12s
2:	learn: 0.5945426	total: 548ms	remaining: 3m 2s
3:	learn: 0.5717426	total: 744ms	remaining: 3m 5s
4:	learn: 0.5561753	total: 945ms	remaining: 3m 8s
5:	learn: 0.5427183	total: 1.09s	remaining: 3m 1s
6:	learn: 0.5314465	total: 1.25s	remaining: 2m 57s
7:	learn: 0.5213462	total: 1.39s	remaining: 2m 52s
8:	learn: 0.5125654	total: 1.55s	remaining: 2m 50s
9:	learn: 0.5059490	total: 1.71s	remaining: 2m 49s
10:	learn: 0.4999545	total: 1.85s	remaining: 2m 46s
11:	learn: 0.4950507	total: 2s	remaining: 2m 44s
12:	learn: 0.4890128	total: 2.14s	remaining: 2m 42s
13:	learn: 0.4836086	total: 2.31s	remaining: 2m 42s
14:	learn: 0.4795392	total: 2.44s	remaining: 2m 40s
15:	learn: 0.4745266	total: 2.6s	remaining: 2m 39s
16:	learn: 0.4689007	total: 2.76s	remaining: 2m 39s
17:	learn: 0.4651921	total: 2.9s	remaining: 2m 38s
18:	learn: 0.4610750	total: 3.06s	remaining: 2m 37s
1

Look at that Average Precision score. We did amazing! 

Actually no, we just overfit. This is likely recurring theme for this competition. It is easy to predict molecules that come from the same corner of chemical space, but generalizing to new molecules is extremely difficult.

## Test Prediction

 The trained Random Forest model is then used to predict the binding probabilities. These predictions are saved to a CSV file, which serves as the submission file for the Kaggle competition.

In [11]:
import os

# Process the test.parquet file chunk by chunk
test_file = '/kaggle/input/leash-predict-chemical-bindings/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=150000):

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = catb_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
